### Semantic Kernel Introduction

#### Overview of Semantic Kernel (SK) and Its Importance
***Semantic Kernel*** is an open-source SDK from Microsoft that acts as middleware between your application code and AI large language models (LLMs). It enables developers to easily integrate AI into apps by letting AI agents call code functions and by orchestrating complex tasks. SK is lightweight and modular, designed for enterprise-grade solutions with features like telemetry and filters for responsible AI. Major companies (including Microsoft) leverage SK because it’s flexible and future-proof – you can swap in new AI models as they emerge without rewriting your code. In short, SK helps build robust, scalable AI applications that can evolve with advancing AI capabilities.

Key reasons why Semantic Kernel is important for AI application development:

- Bridging AI and Code: **SK combines natural language prompts with your existing code and APIs**, allowing AI to take actions. The AI can request a function call and SK will execute that function and return results back to the model. This bridges the gap between what the AI intends and what your code can do.
- **Plugins (Skills)**: You can expose functionalities (from simple math to complex business logic or external APIs) as SK plugins. By describing your code to the AI (via function definitions), the model can invoke these functions to fulfill user requests. This plugin architecture makes your AI solutions modular and extensible.
- **Enterprise-ready**: SK includes support for security, observability, and compliance (e.g. integration with Azure services, monitoring, content filtering). Hooks and filters ensure you can enforce policies (for instance, prevent sensitive data leakage).
- **Multi-modal & Future-Proof**: SK natively supports multiple AI services (OpenAI, Azure OpenAI, HuggingFace, etc.) and modalities. Chat-based APIs can be extended to voice or other modes. As new models (like vision-enabled models or better language models) come out, SK lets you plug them in without major changes.
- **Rapid Development**: By handling the heavy lifting of prompt orchestration, function calling, and memory management, SK enables faster development of AI features. You focus on defining what you want the AI to do (skills, prompts) and SK handles how to do it. Microsoft claims that SK helps “deliver AI solutions faster than any other SDK” due to its ability to automatically call functions.

### Services and Core Components of SK
Semantic Kernel's architecture revolves around a few core components and services:

- **Kernel**: The central object that orchestrates everything. The Kernel holds configuration for AI services, manages plugins (skills), coordinates function calls, and maintains contextual state (memory). You typically create one Kernel instance in your app and use it to register functions and perform AI queries.
- **AI Services**: SK connects to AI models for different tasks:
Chat Models: e.g. Azure OpenAI GPT-4o-mini or GPT-4o for natural language generation and understanding.
Embedding Models: for converting text to vector embeddings (used in memory/search).
Other Modalities: connectors for images, speech, etc., if needed.

Semantic Kernel can automatically read your .env (automatically read from root of project) to access Azure OpenAI using the following variables:

    AZURE_OPENAI_ENDPOINT - The endpoint it should talk to by default
    AZURE_OPENAI_API_KEY - The API Key it should use
    AZURE_OPENAI_API_VERSION - Inference API version it should use per default
    AZURE_OPENAI_CHAT_DEPLOYMENT_NAME - Model deployment name it should use per default

Before you continue, make sure your .env (copied from .env.sample) is filled out correctly.

You configure the Kernel with the endpoints/keys for the services you need. For example, adding an Azure OpenAI chat completion service as follows ->



#### 1. Create a kernel and add a chat completion model to it 

In [1]:
from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

kernel = Kernel()
import dotenv
dotenv.load_dotenv()

# Auto-loads defaults from .env file, alternatively you can set endpoint, deployment_name and api_key directly
chat_completion=AzureChatCompletion()

kernel.add_service(chat_completion)


### 2. Provide the kernel with the required skills. These can be defined as "functions" and "plugins"

##### Define function via a prompt (natural language function)

In [2]:
prompt_template = "{{$input}}\n\ntranslate to {{$target_language}} and change the tone to {{$target_tone}}:"

rewrite_expert = kernel.add_function(
    prompt=prompt_template,
    function_name="toner",
    plugin_name="Toner"
)

# Use the function
sample_text = """
Semantic Kernel is a lightweight, open-source development kit that lets 
you easily build AI agents and integrate the latest AI models into your codebase
"""

# async function call need to be awaited
summary = await kernel.invoke(rewrite_expert, input=sample_text, target_language = "english", target_tone="childish")
print(summary)

Semantic Kernel is a lightweight, open-source development kit that lets you easily build AI agents and integrate the latest AI models into your codebase.

Childish English Translation:  
Semantic Kernel is a super cool, tiny tool you can use for free! It helps you make smart robot friends and put the newest AI magic into your own games and projects—easy peasy!


#### Define function via code 

In [3]:
from typing import TypedDict, Annotated, List, Optional
from semantic_kernel.functions import kernel_function
# Define the LightModel and LightsPlugin classes

class LightModel(TypedDict):
    id: int
    name: str
    is_on: bool | None
    brightness: int | None
    hex: str | None


class LightsPlugin:
    def __init__(self, lights: list[LightModel]):
        self.lights = lights

    @kernel_function
    async def get_lights(self) -> List[LightModel]:
        """Gets a list of lights and their current state."""
        return self.lights

    @kernel_function
    async def get_state(
        self, id: Annotated[int, "The ID of the light"]
    ) -> Optional[LightModel]:
        """Gets the state of a particular light."""
        for light in self.lights:
            if light["id"] == id:
                return light
        return None

    @kernel_function
    async def change_state(
        self, id: Annotated[int, "The ID of the light"], new_state: LightModel
    ) -> Optional[LightModel]:
        """Changes the state of the light."""
        for light in self.lights:
            if light["id"] == id:
                light["is_on"] = new_state.get("is_on", light["is_on"])
                light["brightness"] = new_state.get("brightness", light["brightness"])
                light["hex"] = new_state.get("hex", light["hex"])
                return light
        return None

In [4]:
# Create dependencies for the plugin
# Example with toy data in memory
from semantic_kernel.kernel import Kernel

lights = [
    {"id": 1, "name": "Table Lamp", "is_on": False, "brightness": 100, "hex": "FF0000"},
    {"id": 2, "name": "Porch light", "is_on": False, "brightness": 50, "hex": "00FF00"},
    {"id": 3, "name": "Chandelier", "is_on": True, "brightness": 75, "hex": "0000FF"},
]

plugin = LightsPlugin(lights=lights)
kernel = Kernel()
kernel.add_plugin(
    plugin=plugin,
    plugin_name="Lights",
)


KernelPlugin(name='Lights', description=None, functions={'change_state': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='change_state', plugin_name='Lights', description='Changes the state of the light.', parameters=[KernelParameterMetadata(name='id', description='The ID of the light', default_value=None, type_='int', is_required=True, type_object=<class 'int'>, schema_data={'type': 'integer', 'description': 'The ID of the light'}, include_in_function_choices=True), KernelParameterMetadata(name='new_state', description=None, default_value=None, type_='LightModel', is_required=True, type_object=<class '__main__.LightModel'>, schema_data={'type': 'object', 'properties': {'id': {'type': 'integer'}, 'name': {'type': 'string'}, 'is_on': {'type': ['boolean', 'null']}, 'brightness': {'type': ['integer', 'null']}, 'hex': {'type': ['string', 'null']}}, 'required': ['id', 'name']}, include_in_function_choices=True)], is_prompt=False, is_asynchronous=True, return_parameter=KernelPa

In [5]:

from semantic_kernel.connectors.ai.function_choice_behavior import (
    FunctionChoiceBehavior,
)
from semantic_kernel.contents.chat_history import ChatHistory

from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
    AzureChatPromptExecutionSettings)
import logging

# Enable  logging
# logging.basicConfig(
#     level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
# )

# Enable planning
execution_settings = AzureChatPromptExecutionSettings()
execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# Create a history of the conversation
history = ChatHistory()
user_message = "Please turn on the lamp"
history.add_user_message(user_message)

while(True):
    # Get the response from the AI
    
    result = await chat_completion.get_chat_message_content(
        chat_history=history,
        settings=execution_settings,
        kernel=kernel,
    )

    # Print the results
    print("Assistant > " + str(result))
    if(user_message.lower() == "exit"):
        print("Exiting the chat. Goodbye!")
        break
    # Add the message from the agent to the chat history
    history.add_message(result)

    # Get user input for the next message
    user_message = input("Type another request or say 'exit' to end the chat > ")
    history.add_user_message(user_message)
    if( user_message.lower() == "exit"):
        #autiomatic function calling
        history.add_user_message("Please turn off all the lamps and give me final state of all the lights")




Assistant > The table lamp has been turned on. If you meant a different lamp, please specify which one you'd like to control.
Assistant > All the lamps have been turned off. Here is the final state of all the lights:

- Table Lamp: Off, Brightness 100, Color #FF0000
- Porch light: Off, Brightness 50, Color #00FF00
- Chandelier: Off, Brightness 75, Color #0000FF

Let me know if you need anything else!
Exiting the chat. Goodbye!


### A real-world-like usecase: Contoso, Ltd. 
Let's assume that **Contoso, Ltd.** is an electronic devices distribution company that sells a large variety of devices (ex. headphones, laptops, TVs, etc.) new and refurbished.

##### First, let's set up **the operational** database for Contoso, Ltd.
- You need an Azure postgres db 
- Rename env.sample to .env and add credentials to .env file
- You can run python db_inti.py to set up the database


#### **Example 1**: an inventory analyst working at the sales departments, wants to perform below actions based on current inventory and sales data of the company:
- Get the most updated inventory for a given product name or id
- A list of most sold product names for a given categroy
- Add a new product to the database

This data is stored in a postgreSQL database.

In this example, we take a simple approach as follows:
1. **securely** Connect to the database
2. Load data from required tables from the database when needed
3. Provide all necessary functions and plugins
4. Develop a code using semantic kernel to get the answers

In [7]:
import psycopg2
from src.get_conn import get_connection_uri
from pandas import DataFrame



class Contoso_DataPlugin:
    def __init__(self, db_uri: str):
        self.conn = psycopg2.connect(db_uri)
        self.cursor = self.conn.cursor()
        print("Connected to company's database successfully.")

    @kernel_function
    async def get_product_info(self, product_name: Optional[str] = None, product_id: Optional[int] = None) -> list[dict]:
        """Gets all product information from the database given the name or ID of the product."""
        query = """SELECT 
                    product_id,                   
                    name,
                    inventory,
                    price,
                    refurbished,
                    category
                FROM products
                WHERE (LOWER(name) = LOWER(%(product_name)s) AND %(product_name)s IS NOT NULL)
                   OR (product_id = %(product_id)s AND %(product_id)s IS NOT NULL)
                   """
        if not product_name and not product_id:
            print("No product name or ID provided.")
            return None
        elif product_id:
            self.cursor.execute(query, {"product_name": None, "product_id": product_id})
        else:
            self.cursor.execute(query, {"product_name": product_name, "product_id": None})

            
        rows = self.cursor.fetchall()
        columns = [desc[0] for desc in self.cursor.description]
        try:
            products= DataFrame(rows, columns=columns)
            products.to_dict(orient="records")  # <-- JSON serializabl
            
            return products.to_dict(orient="records")  # <-- JSON serializabl
        except Exception as e:
            print(f"Error fetching product information: {e}")
            return None
    @kernel_function
    def most_sold_product(self, category: str) -> Optional[dict]:
        """Returns the most sold product in a given category."""
        query = """
            SELECT products.product_id, products.name, SUM(sales.quantity) AS total_sold
            FROM sales
            JOIN products ON sales.product_id = products.product_id
            WHERE LOWER(products.category) = LOWER(%(category)s) 
            GROUP BY products.product_id, products.name
            ORDER BY total_sold DESC
            LIMIT 1;
        """
        self.cursor.execute(query, {"category": category})
        row = self.cursor.fetchone()
        if row:
            columns = [desc[0] for desc in self.cursor.description]
            return dict(zip(columns, row))
        else:
            print(f"No sales data found for category: {category}")
            return None
    @kernel_function
    async def add_product(self, name: str, description: str, price: float,
                           inventory: int, refurbished: bool, category: str ) -> bool:
        """Adds a new product to the database."""
        query = """INSERT INTO products (name, description, price, inventory, refurbished, category)
                   VALUES (%(name)s, %(description)s, %(price)s, %(inventory)s, %(refurbished)s, %(category)s);"""  
        try:
            self.cursor.execute(
                query,
                {
                    "name": name,
                    "description": description,
                    "price": price,
                    "inventory": inventory,
                    "refurbished": refurbished,
                    "category": category
                }
            )
            self.conn.commit()
            print(f"Product '{name}' added successfully.")
            return True
        except Exception as e:
            print(f"Error adding product: {e}")
            self.conn.rollback()
            return False

    def close_connection(self):
        """Closes the database connection."""
        self.cursor.close()
        self.conn.close()
        print("Database connection closed.")
    



In [8]:
conn_uri = get_connection_uri()

Connection uri was rertieved successfully.


In [9]:
chat_kernel = Kernel()

# Auto-loads defaults from .env file, alternatively you can set endpoint, deployment_name and api_key directly
chat_completion = AzureChatCompletion()
chat_kernel.add_service(chat_completion)

contoso_plugin = Contoso_DataPlugin(db_uri=conn_uri)

chat_kernel.add_plugin(
    plugin = contoso_plugin,
    plugin_name="contoso_plugin"
) 

# Enable planning
execution_settings = AzureChatPromptExecutionSettings()
execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# Create a history of the conversation
history = ChatHistory()


Connected to company's database successfully.


#### Example 1: asking a question about product

In [10]:
user_message = "Please give me information about the product with id 15"
history.add_user_message(user_message)

# Get the response from the AI
    
result = await chat_completion.get_chat_message_content(
    chat_history=history,
    settings=execution_settings,
    kernel=chat_kernel,
)
print("Assistant > " + str(result))

Assistant > The product with ID 15 is the "Jabra Headphone Studio." Here are the details:

- Inventory: 60 units available
- Price: $179.99
- Refurbished: No (this is a new product)
- Category: Headphones

Let me know if you need more information about this product or anything else!


#### Example 2: adding a new product record

In [11]:
user_message = "add a new product with name 'Lenovo Thinkpad', description 'Latest model', price 699.99, inventory 5, refurbished False, category 'Laptops'"
history.add_user_message(user_message)
result = await chat_completion.get_chat_message_content(
    chat_history=history,
    settings=execution_settings,
    kernel=chat_kernel,
)
print("Assistant > " + str(result))

Product 'Lenovo Thinkpad' added successfully.
Assistant > The product "Lenovo Thinkpad" with the description "Latest model," priced at $699.99, inventory of 5, not refurbished, and in the "Laptops" category, has been successfully added to the database. If you need more details or want to manage this product, let me know!
